In [2]:
import numpy as np
import pandas as pd
import math
import pickle

from sklearn.ensemble import RandomForestClassifier
from sklearn import svm, cross_validation
from sklearn.model_selection import cross_val_score
from sklearn.cross_validation import train_test_split
from sklearn.ensemble import RandomForestRegressor

hedgecolumns=pd.read_csv("columnnames.csv", encoding =  "ISO-8859-1")
hcol=hedgecolumns.values.T.tolist()[0]

C:\Users\hikim.TPR\AppData\Local\Continuum\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
#reading files

##########################################################
##############Insert the file below ######################
##########################################################

#### attach the file name in between " " without any space

hedgeable=pd.read_csv("data_orginal.csv", names=hcol, low_memory=False, skipinitialspace=True)#, skipinitialspace=True, na_values=["?"])


##########################################################
##############Insert the file above ######################
##########################################################

In [4]:

#loading HiJae's Model
HiJae_model= open("HiJae_RF_Model.pkl", "rb")
HiJae_Rf_Model=pickle.load(HiJae_model)

##I just like to attach column names
hedgecolumns=pd.read_csv("columnnames.csv", encoding =  "ISO-8859-1")
hcol=hedgecolumns.values.T.tolist()[0]

In [5]:
#locating non-numeric values so that I can replace them as NaN value 
def isnumber(x):
    try:
        float(x)
        return True
    except:
        return False

#removing null rows in specfic column(s) from a df
def rm_null(df, i):
    return df[df.iloc[:,i].isnull()]

In [6]:
#########Dealing with null values#########

#First, I noticed the null values have string that involves "?"

#I replace non-numeric values with NaN in all attribute values
hedgeable.iloc[:,:-1]=hedgeable.iloc[:,:-1][hedgeable.iloc[:,:-1].applymap(isnumber)]

#The instruction stated that "28% of instances are missing some of the continuous attributes."
#However, one binary column also had 15 null values. 15 out of 3,279 is a small number that I wanted to simply omit.
#I simply omit NaN values in the binary attribute columns & response variable column for the sake of the model
#I wonder how the test set will be
hedgeable=hedgeable.dropna(subset=hedgeable.columns[3:-1], how='any')

In [7]:
##28% of instances are missing some of the continuous attributes.
##Too big to simply omit in this case. (will try later anyways)
##So, I'm gonna RF regress for the missing data

##hedgeable rows without na values
hedgeclean=hedgeable.dropna()
##hedgeable rows only with na values
hedgena=hedgeable[hedgeable.isnull().any(axis=1)]

#getting rows where each column has null values. So I can replace these with predicted values
height=rm_null(hedgeable, 0)    
width =rm_null(hedgeable, 1)  
aratio=rm_null(hedgeable, 2)



In [8]:
#To fill NaN values of the three continuous variables, I am predicting theses values using random forest
#height and weight could use both RF regressor & Classifier
#The minimum number of groups for any class cannot be less than n_splits=3
#So I am trying regressors: svm and rf 

#print(len(df_width_cl.value_counts())) #219
#print(len(df_width_cl.value_counts())) #277

#predictor
df_x_cl=hedgeclean.iloc[:,3:-1] #2359 rows × 1555 columns

#response
df_height_cl=hedgeclean.iloc[:,0] #2359 rows × 1 columns
df_width_cl=hedgeclean.iloc[:,1] #2359 rows × 1 columns
df_aratio_cl=hedgeclean.iloc[:,2] #2359 rows × 1 columns

#splitting test vs train

In [9]:
#tuning tree counts
height_x=height.iloc[:,3:-1]
width_x=width.iloc[:,3:-1]
aratio_x=aratio.iloc[:,3:-1]

#n_estimators have been chosen after cross_validations of each
#height
rf_reg=RandomForestRegressor(n_estimators=120)
rf_reg.fit(df_x_cl, df_height_cl)
height_pred=rf_reg.predict(height_x)

#width
rf_reg=RandomForestRegressor(n_estimators=120)
rf_reg.fit(df_x_cl, df_width_cl)
width_pred=rf_reg.predict(width_x)

#aratio
rf_reg=RandomForestRegressor(n_estimators=80)
rf_reg.fit(df_x_cl, df_aratio_cl)
aratio_pred=rf_reg.predict(aratio_x)

In [10]:
#filling missing values with my predicted values
##the end of data manipulation
hedgeable.loc[hedgeable['height: continuous.'].isnull(), 'height: continuous.']=list(height_pred)
hedgeable.loc[hedgeable['width: continuous.'].isnull(), 'width: continuous.']=list(width_pred)
hedgeable.loc[hedgeable['aratio: continuous.'].isnull(), 'aratio: continuous.']=list(aratio_pred)


In [11]:
x=hedgeable.iloc[:,:-1]
ad=hedgeable.iloc[:,-1]

HiJae_Pred=HiJae_Rf_Model.predict(x)

s=ad.values
correct=0
for i in range(len(HiJae_Pred)):
    if HiJae_Pred[i]==s[i]:
        correct=correct+1
    else:
        correct=correct+0

print("model accuracy : ", correct/len(s))


model accuracy :  0.992953431372549
